In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dev_interaction_util import *
from DevCvAnalysis import DevCvAnalysis

In [3]:
config = load_config("config.yml") 

dropbox_data_dir = config['dropbox_data_dir']


The basic structure is the following:

1. Run the following cross-validated analyses:
   1. Predicting change by condition only
   2. Predicting change by condition and neural and behavioral measures
   3. Predicting change by condition, neural and behavioral measures, and their interactions
2. Measure the predictivity of the three models above using anova
3. Repeat the steps above separately for three outcome variables, change in: FFQ, ASA-24, and BFP
4. 

Tasks to do to get this job done (not in order):

1. Write the analysis pipeline above
2. Get the neural data
3. Get the behavioral data


We have the behavioral data. Do we have the neural data already?

What could we delegate here? Behavioral data we already have. We have mostly writen the analysis pipeline. The neural data could be passed on.

# Get the data

## Set up data


In [4]:
%load_ext autoreload
%autoreload 2

from InterventionCVManager import *

dropbox_data_dir = config['dropbox_data_dir']

icvm = InterventionCVManager(dropbox_data_dir)
icvm.mode = 'full_pipeline_test'

#dev_cv_analysis = icvm.get_prepopulated_dev_cv_analysis(set_as_random=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%load_ext autoreload
%autoreload 2
dev_cv_analysis = icvm.get_prepopulated_dev_cv_analysis(set_as_random=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:918: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV280', 'DEV032', 'DEV022', 'DEV002', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(244, 44)
(244, 44)


# Predict change

### Set up sets of variables to run

First, we set up a function that loops runs the scoring loop above (which does one cross-validation analysis), and the nadditionally:
- selects the best model based on the overall results
- Runs a final fit
- presents model results

Now, we include functions that compare models with and without individual differences and interactions.

Should manually verify that in the following list, the intervention_group allocations are randomized (if we're running a test run) or that they are accurate (if it's not a test run).

In [6]:
pd.concat([
    dev_cv_analysis.outcome_measures['SID'],
      dev_cv_analysis.group_assignments
],axis=1)

,SID,intervention_group
0,DEV001,umpqua
1,DEV004,umpqua
2,DEV005,umpqua
3,DEV008,umpqua
4,DEV009,umpqua
...,...,...
239,DEV308,willamette
240,DEV309,umpqua
241,DEV310,mckenzie
242,DEV311,willamette


In [7]:
dev_cv_analysis.outcome_measures.columns

Index(['SID', 'cancer_promoting_minus_preventing_FFQ', 'cancer_promoting_FFQ',
       'bf', 'NUTRIENT_DENSITY_2wkAverage'],
      dtype='object')

In [8]:
condition_cols = dev_cv_analysis.group_assignment_onehots.columns.tolist()
inddiff_cols = dev_cv_analysis.get_predictors_main_names()

interaction_cols = [id + "*" + cond for id in inddiff_cols for cond in condition_cols]

predictor_sets = {
    'condition_inddiff_interactions': condition_cols + inddiff_cols + interaction_cols,
    'condition_inddiff': condition_cols + inddiff_cols,
    'condition_only': condition_cols
    
    
}


outcome_vars_to_try = [ 'bf','cancer_promoting_FFQ',
       'NUTRIENT_DENSITY_2wkAverage']

In [9]:
print(", ".join(dev_cv_analysis.get_predictor_data().columns))

BSCS, EDM, BIS_11, PCS, RS, TRSQ, ACES_sum, BFI_agreeableness, BFI_conscientiousness, BFI_extraversion, BFI_neuroticism, BFI_openness, IMI_effort_importance, IMI_interest_enjoyment, NCS_total, PLAN_cognitive_strategies, PLAN_mental_flexibility, PLAN_temporal_orientation, RMQ_assessment, TESQ_E_sum, SRHI_healthy_minus_unhealthy, RTFS_f1_minus_f2, cancer_promoting_minus_preventing_craved_FCI, cancer_promoting_minus_preventing_liked_FCI, cSES, age365, education_own, SST_SSD, SST_mean_ssrt_0, ROC_Crave_Regulate_Minus_Look, WTP_unhealthy_minus_healthy, wtp_liked_koban_kober_craving_wmapN99_boot10K_02-May-2022_notzero, wtp_liked_value_association-test_z_FDR_0.01, roc_reappraiseCrave_reappraisal_association-test_z_FDR_0.01, roc_reappraiseCrave_abstract_association-test_z_FDR_0.01, roc_reappraiseCrave_multivariate_regulation, roc_lookCrave_koban_kober_craving_wmapN99_boot10K_02-May-2022_notzero, sst_CorrectGo_striatum_joint_mask, sst_CorrectGo_finger movements_association-test_z_FDR_0.01, sst_

## Body Fat Percentage

In [10]:


model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, outcome_vars_to_try[0], dev_cv_analysis)

## condition_inddiff_interactions

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:918: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV280', 'DEV032', 'DEV022', 'DEV002', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(244, 44)
(244, 44)
 attempting to predict bf with 134 predictors in the set condition_inddiff_interactions
predictors in that set are mckenzie willamette BSCS EDM BIS_11 PCS RS TRSQ ACES_sum BFI_agreeableness BFI_conscientiousness BFI_extraversion BFI_neuroticism BFI_openness IMI_effort_importance IMI_interest_enjoyment NCS_total PLAN_cognitive_strategies PLAN_mental_flexibility PLAN_tempora

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


...

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', RFE(estimator=LinearRegression())),
                ('estimator', Ridge())])
{'feature_selection__n_features_to_select': [2], 'feature_selection__step': [5], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Lasso())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Lasso())])
{'feature_selection__score_func': [<function f_regression at 0x17dbc67a0>], 'feature_selection__k': [2], 'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', 

doing permutation test on importance; this may take time.
Number of selected features: 0


,predictor,coef,feature_importance,fa_abs
0,mckenzie,-0.0,0.0,0.0
1,willamette,-0.0,0.0,0.0


{'condition_inddiff_interactions': -0.12037254665394732, 'condition_inddiff': -0.026836064196948995, 'condition_only': -0.03008587172189068}


## Unhealthy foods

In [11]:


model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'cancer_promoting_FFQ')

## condition_inddiff_interactions

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:918: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV280', 'DEV032', 'DEV022', 'DEV002', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(244, 44)
(244, 44)
 attempting to predict cancer_promoting_FFQ with 134 predictors in the set condition_inddiff_interactions
predictors in that set are mckenzie willamette BSCS EDM BIS_11 PCS RS TRSQ ACES_sum BFI_agreeableness BFI_conscientiousness BFI_extraversion BFI_neuroticism BFI_openness IMI_effort_importance IMI_interest_enjoyment NCS_total PLAN_cognitive_strategies PLAN_mental_flexib

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


...

/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:918: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV280', 'DEV032', 'DEV022', 'DEV002', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(244, 44)
(244, 44)
 attempting to predict cancer_promoting_FFQ with 2 predictors in the set condition_only
predictors in that set are mckenzie willamette
outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler', Stan

doing permutation test on importance; this may take time.
Number of selected features: 0


,predictor,coef,feature_importance,fa_abs
0,mckenzie,0.0,0.0,0.0
1,willamette,0.0,0.0,0.0


{'condition_inddiff_interactions': -0.035586559325716284, 'condition_inddiff': -0.010310857456875055, 'condition_only': -0.02525117228795315}


## Nutrient density

In [12]:


model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'NUTRIENT_DENSITY_2wkAverage')

## condition_inddiff_interactions

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:918: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV280', 'DEV032', 'DEV022', 'DEV002', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(244, 44)
(244, 44)
 attempting to predict NUTRIENT_DENSITY_2wkAverage with 134 predictors in the set condition_inddiff_interactions
predictors in that set are mckenzie willamette BSCS EDM BIS_11 PCS RS TRSQ ACES_sum BFI_agreeableness BFI_conscientiousness BFI_extraversion BFI_neuroticism BFI_openness IMI_effort_importance IMI_interest_enjoyment NCS_total PLAN_cognitive_strategies PLAN_mental

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


...

/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:918: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV280', 'DEV032', 'DEV022', 'DEV002', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
(244, 44)
(244, 44)
 attempting to predict NUTRIENT_DENSITY_2wkAverage with 2 predictors in the set condition_only
predictors in that set are mckenzie willamette
outer split0
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': array([0.1, 1. , 0.2, 0.4, 0.6, 0.8, 1. ])}
Fitting 4 folds for each of 7 candidates, totalling 28 fits
Pipeline(steps=[('scaler

doing permutation test on importance; this may take time.
Number of selected features: 1


,predictor,coef,feature_importance,fa_abs
1,willamette,0.925242,0.012714,0.012714
0,mckenzie,-0.000000,0.000000,0.000000


{'condition_inddiff_interactions': -0.020733343655307566, 'condition_inddiff': -0.21215536349278832, 'condition_only': -0.04479551410013434}
